In [ ]:
import subprocess
from pathlib import Path
from datetime import datetime, timedelta
import re
from collections import defaultdict

def extract_fire_lines(logfile: Path) -> list:
    try:
        result = subprocess.check_output(f"tf -g < {logfile}", shell=True, text=True, stderr=subprocess.DEVNULL)
        return [line for line in result.splitlines()
                if "FUT_TAIFEX_TXF:" in line and "[FIRE]" in line and "[FIRE DONE]" not in line]
    except:
        return []

def extract_all_fire_events(date: str) -> dict:
    base_path = Path("/nfs/datafiles.optiontraderlogs") / date.replace("-", "/")
    logs = {
        "5F": base_path / "capital_neutrino_txf_5f" / f"output.neutrino_txf_5f.{date.replace('-', '')}.log",
        "6F": base_path / "capital_neutrino_txf" / f"output.neutrino_txf.{date.replace('-', '')}.log"
    }

    fire_by_month = defaultdict(list)
    for floor, logfile in logs.items():
        for line in extract_fire_lines(logfile):
            m = re.search(r"FUT_TAIFEX_TXF:(\d+)", line)
            if not m:
                continue
            time_str = line[:15]
            try:
                ts = datetime.strptime(time_str, "%H:%M:%S.%f")
                month = m.group(1)
                fire_by_month[month].append((floor, ts, line))
            except:
                continue

    for v in fire_by_month.values():
        v.sort(key=lambda x: x[1])
    return fire_by_month

def group_matched_fires(fire_by_month: dict, threshold_ms=1) -> dict:
    matched = {}
    for month, events in fire_by_month.items():
        i = 0
        pairs = []
        while i < len(events) - 1:
            f1, t1, l1 = events[i]
            f2, t2, l2 = events[i+1]
            diff = abs((t2 - t1).total_seconds() * 1000)
            if f1 != f2 and diff <= threshold_ms:
                pairs.append([(f1, t1, l1), (f2, t2, l2)])
                i += 2
            else:
                i += 1
        if pairs:
            matched[month] = pairs
    return matched

count_5f_faster = 0
count_6f_faster = 0
total_5f_ms = 0.0
total_6f_ms = 0.0

def print_daily_summary(matched, date_str):
    global count_5f_faster, count_6f_faster, total_5f_ms, total_6f_ms

    print(f"\n\n=== {date_str} ===")
    for month, pairs in matched.items():
        print(f"\n=== Matched FIRE signal for contract {month} ===")
        for g in pairs:
            g = sorted(g, key=lambda x: x[1])
            t1, t2 = g[0][1], g[1][1]
            faster, slower = g[0][0], g[1][0]
            diff_ms = abs((t2 - t1).total_seconds() * 1000)
            print("---")
            print(f"{faster} faster than {slower} by {diff_ms:.3f} ms")
            for f, t, line in g:
                print(f"{f} {t.time()} {line}")
            if faster == "5F":
                count_5f_faster += 1
                total_5f_ms += diff_ms
            else:
                count_6f_faster += 1
                total_6f_ms += diff_ms

def print_final_summary():
    print("\n===== FINAL SUMMARY =====")
    print(f"6F faster count: {count_6f_faster}")
    print(f"6F average faster: {total_6f_ms / count_6f_faster:.3f} ms" if count_6f_faster else "N/A")
    print(f"5F faster count: {count_5f_faster}")
    print(f"5F average faster: {total_5f_ms / count_5f_faster:.3f} ms" if count_5f_faster else "N/A")

def process_fire_range(start_date: str, end_date: str):
    d0 = datetime.strptime(start_date, "%Y-%m-%d").date()
    d1 = datetime.strptime(end_date, "%Y-%m-%d").date()
    while d0 <= d1:
        try:
            fire_by_month = extract_all_fire_events(str(d0))
            matched = group_matched_fires(fire_by_month)
            print_daily_summary(matched, str(d0))
        except Exception as e:
            print(f"\n=== Error on {d0}: {e}")
        d0 += timedelta(days=1)
    print_final_summary()



In [12]:

process_fire_range("2025-06-01", "2025-06-30")




=== 2025-06-01 ===


=== 2025-06-02 ===


=== 2025-06-03 ===


=== 2025-06-04 ===

=== Matched FIRE signal for contract 202506 ===
---
6F faster than 5F by 0.033 ms
6F 08:45:00.100159 08:45:00.100159 FUT_TAIFEX_TXF:202506 [FIRE] dir:sell px:21279 sz:1 m:1@21279x21280@13 bid_fire_offset:0 ask_fire_offset:0 h:4@21265x21270@1 edge:-15x8.25 msglag:0.000112
5F 08:45:00.100192 08:45:00.100192 FUT_TAIFEX_TXF:202506 [FIRE] dir:sell px:21279 sz:1 m:1@21279x21280@13 bid_fire_offset:0 ask_fire_offset:0 h:4@21265x21270@1 edge:-15x8.25 msglag:0.000136
---
6F faster than 5F by 0.023 ms
6F 08:45:00.119394 08:45:00.119394 FUT_TAIFEX_TXF:202506 [FIRE] dir:sell px:21277 sz:1 m:1@21277x21278@1 bid_fire_offset:0 ask_fire_offset:0 h:4@21265x21270@1 edge:-13x6.25 msglag:0.000019
5F 08:45:00.119417 08:45:00.119417 FUT_TAIFEX_TXF:202506 [FIRE] dir:sell px:21278 sz:1 m:2@21278x21279@1 bid_fire_offset:0 ask_fire_offset:0 h:4@21265x21270@1 edge:-14x7.25 msglag:0.000033
---
6F faster than 5F by 0.024 ms
6F 08:4